## Building an image classification model using Keras To Classify

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


### Imports

In [2]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

Using Theano backend.


In [7]:
img_width, img_height = 64, 32

train_data_dir = '../data/train'
validation_data_dir = '../data/validation'
test_data_dir='../data/test'

In [49]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 8599 images belonging to 2 classes.
Found 1843 images belonging to 2 classes.
Found 1844 images belonging to 2 classes.


### Model architecture definition

In [9]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

/Users/surajus/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 32, 3...)`
  
/Users/surajus/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/Users/surajus/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [11]:
nb_epoch = 30
nb_train_samples = 2048
nb_validation_samples = 832

### Training

In [12]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/Users/surajus/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=64, epochs=30, validation_steps=832)`
  


Epoch 1/30
64/64 [==============================] - 33s - loss: 0.1896 - acc: 0.9199 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 2/30
64/64 [==============================] - 33s - loss: 0.0682 - acc: 0.9863 - val_loss: 6.3870e-04 - val_acc: 1.0000
Epoch 3/30
64/64 [==============================] - 33s - loss: 0.0232 - acc: 0.9937 - val_loss: 5.5116e-05 - val_acc: 1.0000
Epoch 4/30
64/64 [==============================] - 32s - loss: 0.0488 - acc: 0.9907 - val_loss: 3.5008e-05 - val_acc: 1.0000
Epoch 5/30
64/64 [==============================] - 32s - loss: 8.1912e-04 - acc: 1.0000 - val_loss: 1.2011e-06 - val_acc: 1.0000
Epoch 6/30
64/64 [==============================] - 33s - loss: 0.1967 - acc: 0.9875 - val_loss: 1.1344e-07 - val_acc: 1.0000
Epoch 7/30
64/64 [==============================] - 32s - loss: 0.4594 - acc: 0.9712 - val_loss: 6.7485 - val_acc: 0.5767
Epoch 8/30
64/64 [==============================] - 33s - loss: 2.5855 - acc: 0.8369 - val_loss: 1.1106e-07 - val_acc: 1.0

In [13]:
model_json = model.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model/model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
model.predict_generator

In [30]:
preprocess = imagenet_utils.preprocess_input
image = load_img("../test_empty.jpg", target_size=(64,32))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = preprocess(image)

In [31]:
preds = model.predict(image)

In [32]:
print preds

[[ 1.]]


In [45]:
preprocess = imagenet_utils.preprocess_input
image1 = load_img("../test_empty.jpg", target_size=(64,32))
image2 = load_img("../test_full.jpg", target_size=(64,32))
image1 = img_to_array(image1)
image2 = img_to_array(image2)
predict_generator = datagen.flow(np.asarray([image1,image2]))

In [51]:
model.predict_generator(predict_generator,2)

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [50]:
model.evaluate_generator(test_generator,1844)

[1.1107657406760619e-07, 1.0]